"""

NOTES:

this data source is an .ods file. Also, it's too long to reliably do with databaker.

For now I've just saved as .xls and used pandas dataframes to restructure it.
Longer term we may need some python to bridge ods->xls so we can pass the filname in as an argument to create
a pipeline.

CONVENTIONS

* oldDf - the "old dataframe". Whatever CSV or xls sheet is loaded at the time.
* newDf - the "new dataframe". A "csv in memory" we're gradually constructing.

"""

In [50]:
# Load the xls. Print the list of sheet names

import pandas as pd

xl = pd.ExcelFile("prc-pfa-mar2013-onwards-tables.xls")
sheets = xl.sheet_names

print(sheets)
print(sheets[-1:])

['Notes_', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18']
['2017-18']


## Build Initial File

Pretty self explanatory. CHOICE_financialTocalendar lets you turn the calendar->financial years conversion on and off as needed.

This will have the dimensions (minus geo code) BUT will also include columns to build the hierarchy (we'll drop them once it's built).

end result is a dataframe variable called 'combinedDf' and some feedback on cube size and sparsity per tab.


In [51]:

CHOICE_financialTocalendar = False           # do we want to hacky-convert to calendar years and quarters


allDataframes = []
for sheetName in sheets[-1:]: # skip sheet index 0 - the Notes tab
    
    # Turn sheet into "old" dataframe. Create new blank dataframe
    oldDf = xl.parse(sheetName)
    newDf = pd.DataFrame()
    
    # Drop the rows with non-place names in "Force Name".
    # Needs to happen now, before we start copying columns

    non_geographic = ["Financial Fraud Action UK", "CIFAS", "Action Fraud", "British Transport Police"]
        
    for ng in non_geographic:
        oldDf = oldDf[oldDf["Force Name"] != ng]
    
    
    # Create a Column: V4_1
    # =========================
    
    # Find whatever they're calling the obs column in this sheet
    if "Force Offences" in oldDf.columns.values:
        offenceCol = "Force Offences"
    elif "Number of Offences" in oldDf.columns.values:
        offenceCol = "Number of Offences"
    else:
        raise ValueError("Cannot identify ONS col. Expecting one of: Number Of Offences, Force Offence. Tab: " +  sheetName)
        
    newDf["V4_0"] = oldDf[offenceCol]
    
    
    # Create Columns: Time | Time_codelist
    # =========================
    if CHOICE_financialTocalendar:
        
        """
        financial-calendar year transform
        
        Example:
        2012/2013 1        2012 Q2
        2012/2013 2        2012 Q3
        2012/2013 3        2012 Q4
        2012/2013 4        2013 Q1
        """
        
        # Start by swapping the quarters column
        LOOKUP_quarters = {1:"Q2", 2:"Q3", 3:"Q4", 4:"Q1"}
        oldDf["Financial Quarter"] = oldDf["Financial Quarter"].map(lambda x: LOOKUP_quarters[x])
        
        # Cut years based on quarters (quarter 1 is the year after the / in "YYYY/YY", the others are before)
        oldDf["Financial Year"][oldDf["Financial Quarter"] == "Q1"] = oldDf["Financial Year"].map(lambda x: x[:2] + x.split("/")[1][-2:])
        oldDf["Financial Year"][oldDf["Financial Quarter"] != "Q1"] = oldDf["Financial Year"].map(lambda x: x[:4])

        newDf["Time_codelist"] = "Quarters"
        newDf["Time"] = oldDf["Financial Year"] + ' ' + oldDf["Financial Quarter"]
        
    else:
        
        # Take time as-is but add the Q to quarter
        newDf["Time_codelist"] = "Quarters"
        newDf["Time"] = oldDf["Financial Year"] + " " + oldDf["Financial Quarter"].map(lambda x: "Q" + str(x))
        

    # Create Columns: Geography | Geography_codelist
    # =========================

    newDf["Geography_codelist"] = ""
    newDf["Geography"] = oldDf["Force Name"]

    
    # Create Columns: Offence | Offence_codelist
    # =========================
    
    newDf["Offence_codelist"] = oldDf["Offence Code"]
    newDf["Offence"] = oldDf["Offence Description"]
        
        
    # Data we'll need to build hierarchy (will drop it before we output the V4)
    newDf["Offence Group"] = oldDf["Offence Group"]
    newDf["Offence Subgroup"] = oldDf["Offence Subgroup"]
    newDf["Offence Description"] = oldDf["Offence Description"]
                
    allDataframes.append(newDf)


# Combine all
combinedDf = pd.concat(allDataframes)

combinedDf.columns.values

array(['V4_0', 'Time_codelist', 'Time', 'Geography_codelist', 'Geography',
       'Offence_codelist', 'Offence', 'Offence Group', 'Offence Subgroup',
       'Offence Description'], dtype=object)

---

## Get Geography Codes

Have used the"police force areas" csv from the Open Geography Portal to look these up.
http://geoportal.statistics.gov.uk/datasets/police-force-areas-december-2017-names-and-codes-in-the-united-kingdom

included in repo as "Police_Force_Areas_December_2016.csv

In [52]:

policeAreasCSV = pd.read_csv("Police_Force_Areas_December_2016.csv")

# sample 5 lines
policeAreasCSV[:5]

,PFA16CD,PFA16NM,FID
0,E23000001,Metropolitan Police,1
1,E23000002,Cumbria,2
2,E23000003,Lancashire,3
3,E23000004,Merseyside,4
4,E23000005,Greater Manchester,5


In [53]:

# cut the blanks at the end
policeAreasCSV.fillna("", inplace=True)
policeAreasCSV = policeAreasCSV[policeAreasCSV["PFA16NM"] != ""]

# deal with idiotic rephrasing
policeAreasCSV["PFA16NM"] = policeAreasCSV["PFA16NM"].map(lambda x: x.replace("City of London","London, City of"))

# build a dict/map to lookup
keys = list(policeAreasCSV["PFA16NM"].unique())
values = list(policeAreasCSV["PFA16CD"].unique())
policeLookup = dict(zip(keys, values))

# Insert the codes
combinedDf["Geography_codelist"] = combinedDf["Geography"].map(lambda x: policeLookup[x])

# Sanity check
combinedDf["Geography_codelist"].unique()

array(['E23000036', 'E23000026', 'E23000023', 'E23000006', 'E23000013',
       'E23000002', 'E23000018', 'E23000035', 'E23000039', 'E23000008',
       'W15000004', 'E23000028', 'E23000037', 'E23000005', 'W15000002',
       'E23000030', 'E23000027', 'E23000012', 'E23000032', 'E23000003',
       'E23000021', 'E23000020', 'E23000034', 'E23000004', 'E23000001',
       'E23000024', 'W15000001', 'E23000009', 'E23000022', 'E23000007',
       'E23000019', 'W15000003', 'E23000011', 'E23000015', 'E23000025',
       'E23000031', 'E23000033', 'E23000029', 'E23000017', 'E23000016',
       'E23000014', 'E23000010', 'E23000038'], dtype=object)


---

## Build Offences hierarchy

NOTE - offences is presented in a semi-hierarcical way. With only the children having data and the layers above 
only being used to categorise. Will insert holding-codes for the upper layers (shouldn't matter since they're not 
exposed) and build it anyway to see what it looks like.


In [54]:
"""

Using the columns:
offence Description | Offence Group | Offence Subgroup | Offence Code

We need to build a CSV with the structure:
Codelist | Code | Label| ParentCode

so we can use build a cypher file with:
https://github.com/ONSdigital/dp-hierarchy-builder/tree/cmd-develop/cmd/hierarchy-transformer
"""

codeList = "offence"

# Initialise with a generic parent

hierarchyCSV = {
    "Codelist":[codeList],
    "Code":["CrimeParent"],
    "Label":["Offences"],
    "ParentCode":[" "]
}


LOOKUP_OG = {}
# Create codes for the top level "Offence Group"
for OG in combinedDf["Offence Group"].unique():
    LOOKUP_OG.update({OG:OG.replace(" ", "-").lower().strip()})
    
# Sanity check dict/map
from pprint import pprint
pprint(LOOKUP_OG)
    

{'Criminal damage and arson': 'criminal-damage-and-arson',
 'Drug offences': 'drug-offences',
 'Fraud offences': 'fraud-offences',
 'Miscellaneous crimes against society': 'miscellaneous-crimes-against-society',
 'Possession of weapons offences': 'possession-of-weapons-offences',
 'Public order offences': 'public-order-offences',
 'Robbery': 'robbery',
 'Sexual offences': 'sexual-offences',
 'Theft offences': 'theft-offences',
 'Violence against the person': 'violence-against-the-person'}


In [55]:

# Now add them to our hierarhcyCSV and sanity check it too
hierarchyCSV = {
    "Codelist":[codeList],
    "Code":["Offence"],
    "Label":["Offence"],
    "ParentCode":[""]
}

for offence in LOOKUP_OG:
    hierarchyCSV["Codelist"].append(codeList)
    hierarchyCSV["Code"].append(LOOKUP_OG[offence])
    hierarchyCSV["Label"].append(offence)
    hierarchyCSV["ParentCode"].append("offence")
    

# Make dataframe and sanity check
newDf = pd.DataFrame.from_dict(hierarchyCSV)
newDf
    

,Code,Codelist,Label,ParentCode
0,Offence,offence,Offence,
1,miscellaneous-crimes-against-society,offence,Miscellaneous crimes against society,offence
2,sexual-offences,offence,Sexual offences,offence
3,theft-offences,offence,Theft offences,offence
4,criminal-damage-and-arson,offence,Criminal damage and arson,offence
5,violence-against-the-person,offence,Violence against the person,offence
6,drug-offences,offence,Drug offences,offence
7,possession-of-weapons-offences,offence,Possession of weapons offences,offence
8,public-order-offences,offence,Public order offences,offence
9,robbery,offence,Robbery,offence


In [56]:

# Going to have to iterate and compare for next level. this will be slow

# We need to track each label so we only add it once.
labelsAlreadySeen = []
LOOKUP_SG = {}

for index, row in combinedDf.iterrows():
    
    if row["Offence Subgroup"] not in labelsAlreadySeen:
        
        # will need to code-ify the label again
        code = row["Offence"].replace(" ", "-").lower().strip()
        
        hierarchyCSV["Codelist"].append(codeList)
        hierarchyCSV["Code"].append(code)
        hierarchyCSV["Label"].append(row["Offence Subgroup"])
        hierarchyCSV["ParentCode"].append(LOOKUP_OG[row["Offence Group"]])
        
        labelsAlreadySeen.append(row["Offence Subgroup"])
        
        # Will need another lookup dict/map for last stage
        LOOKUP_SG.update({row["Offence Subgroup"]:code})

# Sanity check. Use last 10 rows
newDf = pd.DataFrame.from_dict(hierarchyCSV)
newDf[-10:]



,Code,Codelist,Label,ParentCode
26,other-offences-against-the-state-or-public-order,offence,Public order offences,public-order-offences
27,rape-of-a-female-aged-16-and-over,offence,Rape,sexual-offences
28,robbery-of-business-property,offence,Robbery of business property,robbery
29,robbery-of-personal-property,offence,Robbery of personal property,robbery
30,shoplifting,offence,Shoplifting,theft-offences
31,theft-from-the-person,offence,Theft from the person,theft-offences
32,theft-from-vehicle,offence,Theft from a vehicle,theft-offences
33,theft-or-unauthorised-taking-of-a-pedal-cycle,offence,Bicycle theft,theft-offences
34,trafficking-in-controlled-drugs,offence,Trafficking of drugs,drug-offences
35,bankruptcy-and-insolvency-(outcomes-only),offence,Fraud offences to 2012/13,fraud-offences


In [57]:

# We need to track each label so we only add it once.
# AND we also need to make sure each label:code combination is consistant.
labelsAlreadySeen = {}
codesAlreadySeen = {}
issuesLog = []

for index, row in combinedDf.iterrows():
    
        # Strip any trailing spaces etc in code
        code = row["Offence_codelist"]
        code = str(code).strip()

        hierarchyCSV["Codelist"].append(codeList)
        hierarchyCSV["Code"].append(code)
        hierarchyCSV["Label"].append(row["Offence"])
        hierarchyCSV["ParentCode"].append(LOOKUP_SG[row["Offence Subgroup"]])
        
        # Throw an error is the same label is present with mutiple representative codes
        # .... this is how we knew about Homicide (see above)
        
        if row["Offence"] not in labelsAlreadySeen.keys():
            
            # Make sure we havn't had this code already for another label
            if code in codesAlreadySeen.keys():
                errorOut = """
                A single value has multiple codes
                {code}
                {a}
                {b}
                """.format(code=code,
                           a=row["Offence"] ,
                           b=codesAlreadySeen[code])
                if errorOut not in issuesLog:
                    issuesLog.append(errorOut)
                
            labelsAlreadySeen.update({row["Offence"]:code})
            codesAlreadySeen.update({code:row["Offence"]})
            
        else:
 
            # We've seen if before. Check that it matching to the same code.
            if labelsAlreadySeen[row["Offence"]] != code:
                errorOut = """
                A single code use for multiple vales:
                {label}
                {a}
                {b}
                """.format(label=row["Offence"],
                           a=labelsAlreadySeen[row["Offence"]], 
                           b=row["Offence_codelist"])
                if errorOut not in issuesLog:
                    issuesLog.append(errorOut)

if len(issuesLog) > 0:
    print("Error: There is not a 1-to-1 relationship between desciption and code")
    for issue in issuesLog:  
        print(issue)
else:
    print("Codes are fine.")
    
    # Output our hierarchy file
    newDf = pd.DataFrame.from_dict(hierarchyCSV)
    
    # Restrcture for order
    # TODO - in place!
    outFile = pd.DataFrame()
    outFile["Codelist"] = newDf["Codelist"]
    outFile["Code"] = newDf["Code"]
    outFile["Label"] = newDf["Label"]
    outFile["ParentCode"] = newDf["ParentCode"]
    
    import csv
    
    outFile.to_csv("Hirarchy_CSV_Offences.csv", quoting=csv.QUOTE_ALL, index=False)
    
    # Sanity check, first 20 rows
    newDf[:20]
    

Codes are fine.


In [58]:

"""
Combine each dataframe into once big one. Then check sparsity and output to csv
"""

# Drop the unwanted columns we only needed for building the hierarchy
unWanted = ["Offence Description","Offence Group","Offence Subgroup","Offence Code"]
for column in combinedDf.columns.values:
    if column in unWanted:
        combinedDf = combinedDf.drop(column, axis=1)

# Check all the sparsity:
d1 = len(combinedDf["Offence"].unique())
d2 = len(combinedDf["Geography"].unique())
d3 = len(combinedDf["Time"].unique())
cubeSize = d1 * d2 * d3
    
print("COMBINED CUBE")
print("Logical Cube Size: ", cubeSize)
print("Number of rows:    ", len(combinedDf))
print("Sparsity:          ", 100 - ((100/cubeSize)*len(combinedDf)))
print("")

combinedDf.columns.values

COMBINED CUBE
Logical Cube Size:  11696
Number of rows:     11696
Sparsity:           0.0



array(['V4_0', 'Time_codelist', 'Time', 'Geography_codelist', 'Geography',
       'Offence_codelist', 'Offence'], dtype=object)

In [59]:
"""
CMD-ify column names
"""

columnNameChanges = {
    'Time_codelist':'calendar-years',
    'Time':'time',
    'Geography_codelist':'police-force-geography',
    'Geography':'geography',
    'Offence_codelist':'offence_codelist',
    'Offence':'offence',
}

newHeaders = []
for col in combinedDf:
    if col in columnNameChanges:
        col = columnNameChanges[col]
    newHeaders.append(col)
    
combinedDf.columns = newHeaders


In [60]:
"""
Remove non cmd friendly special characters from 
"""

combinedDf["offence_codelist"] = combinedDf["offence_codelist"].map(lambda x: x.replace("/", "-"))

combinedDf["offence_codelist"].unique()

array(['80', '71', '73', '31A', '31', '29', '29A', '37.2', '76', '56A',
       '56B', '8N', '8S', '5D', '105A', '104', '30D', '30B', '28B', '28F',
       '28D', '28H', '2', '83', '26', '35', '30C', '30A', '28A', '28E',
       '37.1', '4.6', '4.8', '4.4', '4.9', '4.7', '22A', '13', '15', '3A',
       '58B', '58A', '58C', '11A', '802', '95', '43', '28C', '28G', '5E',
       '24', '88E', '60', '814', '33', '54', '8L', '1-4.1-4.10-4.2', '23',
       '4.3', '126', '36', '49A', '33A', '8R', '106', '86', '69', '58D',
       '92C', '81', '61', '90', '88C', '99', '66', '49', '67', '79', '10D',
       '92E', '92D', '61A', '10B', '10A', '10C', '14', '38', '9A', '8P',
       '105B', '58J', '8M', '9B', '19C', '19E', '19D', '19F', '19H', '19G',
       '34A', '34B', '70', '21', '22B', '20A', '20B', '17A', '17B', '88A',
       '46', '27', '8Q', '41', '47', '39', '45', '40', '42', '44', '48',
       '59', '3B', '72', '92A', '88D', '62A', '96', '55', '52', '53F',
       '51', '53E', '53C', '53D', '53B']

In [61]:
#newDf.to_csv("lastSample.csv", index=False)

if CHOICE_financialTocalendar:
    outName = "CALENDAR_CimeWithHomeOffice.csv"
else:
    outName = "FINANCE_CimeWithHomeOffice.csv"
    
combinedDf.to_csv(outName, index=False)
print("CSV Written to V4")


combinedDf.columns.values

CSV Written to V4


array(['V4_0', 'calendar-years', 'time', 'police-force-geography',
       'geography', 'offence_codelist', 'offence'], dtype=object)

---

## Build codelists

Build simple codelists



In [62]:

# geography
codelist = pd.DataFrame()
codelist["geography"] = combinedDf["geography"]
codelist["police-force-geography"] = combinedDf["police-force-geography"]
codelist = codelist.drop_duplicates()
codelist.to_csv("CL_geography_codelist.csv", index=False)


# offence
codelist = pd.DataFrame()
codelist["offence"] = combinedDf["offence"]
codelist["offence_codelist"] = combinedDf["offence_codelist"]
codelist = codelist.drop_duplicates()
codelist.to_csv("CL_offence_codelist.csv", index=False)
